In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [2]:
#Dataset on restaurants in california that serve burritos
data = pd.read_csv('C:/Users/varun/Downloads/burritos.csv')

In [3]:
#Raw data
data.head()

,Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,...,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
0,Donato's taco shop,California,1/18/2016,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,NaN,6.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Oscar's Mexican food,California,1/24/2016,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,NaN,5.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Oscar's Mexican food,Carnitas,1/24/2016,NaN,NaN,NaN,NaN,NaN,NaN,4.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oscar's Mexican food,Carne asada,1/24/2016,NaN,NaN,NaN,NaN,NaN,NaN,5.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pollos Maria,California,1/27/2016,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Data from Kaggle
data.shape

(385, 66)

In [5]:
#Attributes considered for the analysis
df = data[['Location','Neighborhood','Address','Yelp','Google','Cost']]
df.shape

(385, 6)

In [6]:
#removing duplicate rows that have the same location data
df = df.drop_duplicates(subset ='Location',keep = 'first')
df.shape

(103, 6)

In [7]:
df = df.reset_index(drop=True)
df.head(10)

,Location,Neighborhood,Address,Yelp,Google,Cost
0,Donato's taco shop,Miramar,6780 Miramar Rd,3.5,4.2,6.49
1,Oscar's Mexican food,San Marcos,225 S Rancho Santa Fe Rd,3.5,3.3,5.45
2,Pollos Maria,Carlsbad,3055 Harding St,4.0,3.8,6.59
3,Nico's Taco Shop,Carmel Valley,3860 Valley Centre Dr #404,3.0,2.9,7.19
4,Los Primos Mexican Food,UTC,7770 Regents Rd,3.0,3.7,9.25
5,JV's Mexican Food,Morena,1112 Morena Blvd,4.0,4.1,6.25
6,Tony's Fresh Mexican Food,Miramar,7122 Miramar Rd,3.0,4.1,6.95
7,Taco Stand,La Jolla,621 Pearl St,4.5,4.4,7.49
8,Graciela's Taco Shop,Bonita,5047 Central Ave,4.0,4.5,5.99
9,Rigoberto's Taco Shop La Jolla,La Jolla,7345 La Jolla Blvd,2.5,4.0,6.40


In [8]:
# Extracting only the neighborhood coloumn of the data for further analysis
df_data = df[['Neighborhood']]
df_data

,Neighborhood
0,Miramar
1,San Marcos
2,Carlsbad
3,Carmel Valley
4,UTC
5,Morena
6,Miramar
7,La Jolla
8,Bonita
9,La Jolla


In [9]:
#setting up foursquare credentials
CLIENT_ID = 'A33KNABNTIUF2HZ4CGMKKWI3OAX5WP4MKCVLOWMRHFJ2PPUC' # your Foursquare ID
CLIENT_SECRET = 'OC14T0UDLSRMPFAEK1NBSP52CGGW3YORYF4NJ31WPFMIHAEO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A33KNABNTIUF2HZ4CGMKKWI3OAX5WP4MKCVLOWMRHFJ2PPUC
CLIENT_SECRET:OC14T0UDLSRMPFAEK1NBSP52CGGW3YORYF4NJ31WPFMIHAEO


In [10]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, san diego,California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [12]:
from geopy.geocoders import Nominatim # to get the latitude and logitude of the neighborhood 
import geocoder # to get coordinates

In [13]:
coords = [ get_latlng(neighborhood) for neighborhood in df_data["Neighborhood"].tolist() ]

In [14]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
df_data['Latitude'] = df_coords['Latitude']
df_data['Longitude'] = df_coords['Longitude']

C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
print(df_data.shape)
df_data

(103, 3)


,Neighborhood,Latitude,Longitude
0,Miramar,32.893610,-117.133400
1,San Marcos,33.140830,-117.160160
2,Carlsbad,33.165880,-117.338220
3,Carmel Valley,32.937830,-117.230860
4,UTC,32.877150,-117.237430
5,Morena,32.769490,-117.190070
6,Miramar,32.893610,-117.133400
7,La Jolla,32.840850,-117.257800
8,Bonita,32.674180,-117.017200
9,La Jolla,32.840850,-117.257800


In [18]:
df_data.to_csv("kl_df.csv", index=False)

In [19]:
# to get coordinates of san diego, california
address = 'san diego, California'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of san diego, california {}, {}.'.format(latitude, longitude))

The geograpical coordinate of san diego, california 32.7174209, -117.1627714.


In [25]:
CLIENT_ID = 'A33KNABNTIUF2HZ4CGMKKWI3OAX5WP4MKCVLOWMRHFJ2PPUC' #  Foursquare ID
CLIENT_SECRET = 'OC14T0UDLSRMPFAEK1NBSP52CGGW3YORYF4NJ31WPFMIHAEO' #  Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A33KNABNTIUF2HZ4CGMKKWI3OAX5WP4MKCVLOWMRHFJ2PPUC
CLIENT_SECRET:OC14T0UDLSRMPFAEK1NBSP52CGGW3YORYF4NJ31WPFMIHAEO


In [70]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [92]:
# Creating a new data frame with venues
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(9207, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Miramar,32.89361,-117.1334,Punjabi Tandoor,32.896648,-117.128265,Indian Restaurant
1,Miramar,32.89361,-117.1334,Mikkeller Brewing San Diego,32.892377,-117.144789,Brewery
2,Miramar,32.89361,-117.1334,Flying Leatherneck Aviation Museum,32.891282,-117.138247,Museum
3,Miramar,32.89361,-117.1334,Miramar Cash & Carry,32.894746,-117.128413,Grocery Store
4,Miramar,32.89361,-117.1334,At Ease Games,32.895411,-117.132148,Hobby Shop
5,Miramar,32.89361,-117.1334,White Labs Brewing Co.,32.895798,-117.121599,Brewery
6,Miramar,32.89361,-117.1334,AleSmith Brewing Company,32.887830,-117.149521,Brewery
7,Miramar,32.89361,-117.1334,Float Spa San Diego,32.895711,-117.127396,Spa
8,Miramar,32.89361,-117.1334,That's Amore Pizza,32.894984,-117.128035,Pizza Place
9,Miramar,32.89361,-117.1334,Marine Corps Exchange,32.891849,-117.123241,Department Store


In [72]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Austin,100,100,100,100,100,100
Bonita,73,73,73,73,73,73
Bonsall,15,15,15,15,15,15
Cardiff,69,69,69,69,69,69
Carlsbad,300,300,300,300,300,300
Carmel Valley,200,200,200,200,200,200
Clairemont,900,900,900,900,900,900
Del Mar,200,200,200,200,200,200
Downtown,200,200,200,200,200,200


In [73]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 283 uniques categories.


In [94]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:283]

array(['Indian Restaurant', 'Brewery', 'Museum', 'Grocery Store',
       'Hobby Shop', 'Spa', 'Pizza Place', 'Department Store',
       'Mediterranean Restaurant', 'Thai Restaurant', 'Deli / Bodega',
       'Government Building', 'Gym / Fitness Center', 'Sandwich Place',
       'Gas Station', 'Golf Course', 'Bakery', 'Sushi Restaurant',
       'Mexican Restaurant', 'Vietnamese Restaurant', 'Coffee Shop',
       'Seafood Restaurant', 'Juice Bar', 'Japanese Restaurant',
       'Burger Joint', 'Smoothie Shop', 'Asian Restaurant', 'Taco Place',
       'Breakfast Spot', 'Fast Food Restaurant', 'Dessert Shop',
       'Korean Restaurant', 'Movie Theater', 'Diner',
       'Italian Restaurant', 'Salad Place', 'Bookstore', 'Hotel',
       'Hardware Store', 'Gastropub', 'Greek Restaurant', 'BBQ Joint',
       'Park', 'Marijuana Dispensary', 'Big Box Store', 'Supermarket',
       'Pet Store', 'Supplement Shop', 'Miscellaneous Shop',
       'Mobile Phone Shop', 'Ice Cream Shop', 'Bubble Tea Shop',


In [95]:
# check if the results contain "American Restaurant"
"American Restaurant" in venues_df['VenueCategory'].unique()

True

In [97]:
# one hot encoding
sd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sd_onehot.columns[-1]] + list(sd_onehot.columns[:-1])
sd_onehot = sd_onehot[fixed_columns]

print(kl_onehot.shape)
sd_onehot.head()

(9207, 284)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Cafeteria,College Gym,College Library,College Rec Center,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Park,Pedestrian Plaza,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings J

In [98]:
sd_grouped = sd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sd_grouped.shape)
sd_grouped

(39, 284)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Cafeteria,College Gym,College Library,College Rec Center,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Park,Pedestrian Plaza,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings J

In [99]:
len(sd_grouped[sd_grouped["American Restaurant"] > 0])

31

In [100]:
sd_ar = sd_grouped[["Neighborhoods","American Restaurant"]]
sd_ar.head()

,Neighborhoods,American Restaurant
0,Austin,0.020000
1,Bonita,0.000000
2,Bonsall,0.066667
3,Cardiff,0.000000
4,Carlsbad,0.060000


In [101]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

sd_clustering = sd_ar.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 1, 0, 2, 1, 0, 2, 2])

In [103]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sd_merged = sd_ar.copy()

# add clustering labels
sd_merged["Cluster Labels"] = kmeans.labels_

In [104]:
sd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sd_merged.head()

,Neighborhood,American Restaurant,Cluster Labels
0,Austin,0.020000,2
1,Bonita,0.000000,1
2,Bonsall,0.066667,0
3,Cardiff,0.000000,1
4,Carlsbad,0.060000,0


In [105]:
sd_merged = sd_merged.join(df_data.set_index("Neighborhood"), on="Neighborhood")

print(sd_merged.shape)
sd_merged.head() 

(84, 5)


,Neighborhood,American Restaurant,Cluster Labels,Latitude,Longitude
0,Austin,0.020000,2,32.764879,-117.081126
1,Bonita,0.000000,1,32.674180,-117.017200
2,Bonsall,0.066667,0,33.286210,-117.245030
3,Cardiff,0.000000,1,32.712753,-117.024515
4,Carlsbad,0.060000,0,33.165880,-117.338220


In [106]:
print(sd_merged.shape)
sd_merged.sort_values(["Cluster Labels"], inplace=True)
sd_merged

(84, 5)


,Neighborhood,American Restaurant,Cluster Labels,Latitude,Longitude
24,Oceanside,0.080000,0,33.197150,-117.380580
24,Oceanside,0.080000,0,33.197150,-117.380580
11,Hillcrest,0.060000,0,32.749950,-117.165560
30,Solana Beach,0.060000,0,32.987350,-117.270690
16,Linda Vista,0.080000,0,32.784190,-117.171350
16,Linda Vista,0.080000,0,32.784190,-117.171350
7,Del Mar,0.070000,0,32.955260,-117.263620
7,Del Mar,0.070000,0,32.955260,-117.263620
11,Hillcrest,0.060000,0,32.749950,-117.165560
20,Morena,0.050000,0,32.769490,-117.190070


In [107]:
sd_merged.loc[sd_merged['Cluster Labels'] == 0]

,Neighborhood,American Restaurant,Cluster Labels,Latitude,Longitude
24,Oceanside,0.080000,0,33.19715,-117.38058
24,Oceanside,0.080000,0,33.19715,-117.38058
11,Hillcrest,0.060000,0,32.74995,-117.16556
30,Solana Beach,0.060000,0,32.98735,-117.27069
16,Linda Vista,0.080000,0,32.78419,-117.17135
16,Linda Vista,0.080000,0,32.78419,-117.17135
7,Del Mar,0.070000,0,32.95526,-117.26362
7,Del Mar,0.070000,0,32.95526,-117.26362
11,Hillcrest,0.060000,0,32.74995,-117.16556
20,Morena,0.050000,0,32.76949,-117.19007


In [108]:
sd_merged.loc[sd_merged['Cluster Labels'] == 1]

,Neighborhood,American Restaurant,Cluster Labels,Latitude,Longitude
12,Houston,0.00,1,32.757471,-117.208321
13,Kearny Mesa,0.01,1,32.821810,-117.149620
13,Kearny Mesa,0.01,1,32.821810,-117.149620
25,Pacific Beach,0.01,1,32.801390,-117.241360
25,Pacific Beach,0.01,1,32.801390,-117.241360
17,Miramar,0.00,1,32.893610,-117.133400
17,Miramar,0.00,1,32.893610,-117.133400
17,Miramar,0.00,1,32.893610,-117.133400
18,Miramar,0.00,1,32.893610,-117.133400
25,Pacific Beach,0.01,1,32.801390,-117.241360


In [109]:
sd_merged.loc[sd_merged['Cluster Labels'] == 2]


,Neighborhood,American Restaurant,Cluster Labels,Latitude,Longitude
35,University City,0.02,2,32.851440,-117.216390
36,University Heights,0.04,2,32.759070,-117.148160
28,Santa Monica,0.02,2,32.744641,-117.245319
33,Temecula,0.02,2,32.751322,-117.230939
26,San Diego,0.03,2,32.715680,-117.161710
27,San Marcos,0.02,2,33.140830,-117.160160
27,San Marcos,0.02,2,33.140830,-117.160160
27,San Marcos,0.02,2,33.140830,-117.160160
29,Seattle,0.02,2,32.751547,-117.034092
27,San Marcos,0.02,2,33.140830,-117.160160


In [88]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")



Libraries imported.


In [89]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

In [90]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters